In [4]:
#######################################################
###                                                 ###
###   /!\  RUN DL_LoadProduct1.ipynb BEFORE   /!\   ###
###                                                 ###
#######################################################

In [4]:
library(jsonlite)
library(rlist)
library(data.table)
suppressPackageStartupMessages(library("dplyr"))
library(XML)
library(methods)

In [5]:
load(file = "all_active_clinical_entities.Rdata")

In [6]:
number_all_active_clinical_entities=length(all_active_clinical_entities)
number_all_active_clinical_entities

[1] 9339

In [7]:
### Number of disorders
indice_list=1
disorders=list()
for (i in 1:length(all_active_clinical_entities))
  if (all_active_clinical_entities[[i]]$DisorderGroup$.attrs=='36547')
  {
    disorders[[indice_list]]=all_active_clinical_entities[[i]]
    indice_list=indice_list+1
  }

In [8]:
# We keep the OrphaCodes of the disorders
orphacode_disorders=list()
for (i in 1:length(disorders))
{
  orphacode_disorders[[i]]=data.frame(disorders[[i]]$OrphaCode)
}
orphacode_disorders=rbindlist(orphacode_disorders)
names(orphacode_disorders)=c('OrphaCode')

In [9]:
# We select the xmls with the datas we need
setwd('/home/jovyan/xml/')

In [10]:
# We can now work on preferential parents
# We select the xml with the datas we need

### NEED TO SEE WHY ALL PRODUCTS 7 LANGS !!!!!!

xmlfiles = list.files(pattern='product7.xml')
product7=xmlToList(xmlfiles)

# Our list for working
product7=product7$DisorderList
indice_list=1
product7_list=list()
for (i in 1:length(product7))
{
  product7_list[[indice_list]]=product7[[i]]
  indice_list=indice_list+1
}
# Each element in the list is a disorder

In [11]:
# We keep only the disorders which have a preferential parent
indice_list=1
disorders_with_preferential_parent=list()
for (i in 1:(length(product7_list)-1))
  if ('TargetDisorder' %in% names(product7_list[[i]]$DisorderDisorderAssociationList$DisorderDisorderAssociation))
  {
    disorders_with_preferential_parent[[indice_list]]=data.frame(product7_list[[i]]$OrphaCode, 
                                                                 product7_list[[i]]$DisorderDisorderAssociationList$DisorderDisorderAssociation$TargetDisorder$Name$text)
    indice_list=indice_list+1
  }
disorders_with_preferential_parent=rbindlist(disorders_with_preferential_parent,use.names = TRUE)
names(disorders_with_preferential_parent)=c('OrphaCode', 'Preferential parent')

In [12]:
# We keep only those which are active disorders
active_preferential_parent=merge(orphacode_disorders, disorders_with_preferential_parent,
                                 by='OrphaCode', all.x = TRUE)

In [13]:
### Distribution of preferential parents
distribution_disorders_by_preferential_parent=as.data.frame(table(active_preferential_parent$`Preferential parent`))
names(distribution_disorders_by_preferential_parent)=c('Preferential parent','Distribution')
distribution_disorders_by_preferential_parent

Preferential parent,Distribution
<fct>,<int>
Rare abdominal surgical disease,7
Rare bone disease,354
Rare cardiac disease,46
Rare circulatory system disease,19
Rare developmental defect during embryogenesis,1968
Rare disorder due to toxic effects,25
Rare disorder potentially indicated for transplant or complication after transplantation,2
Rare endocrine disease,185
Rare gastroenterologic disease,74
